In [114]:
import pandas as pd

# Read the News Category Dataset v3 (newline-delimited JSON)
df = pd.read_json('News_Category_Dataset_v3.json', lines=True)

# Save it as CSV
df.to_csv('News_Category_Dataset_v3.csv', index=False)

print("Conversion successful. CSV saved.")


Conversion successful. CSV saved.


In [115]:
data=df.iloc[:,1].values
x=data.tolist()



#x
#x[150302]

In [116]:
len(x)

209527

In [117]:
import pandas as pd
import numpy as np

# Convert to Series
x_series = pd.Series(x)

# Drop None and NaN
x_cleaned = x_series.dropna()

# Drop empty and whitespace-only strings
x_cleaned = x_cleaned[~x_cleaned.astype(str).str.strip().eq("")]

# Update the original list
x = x_cleaned.tolist()

            
len(x)  # (updated size)


209521

In [118]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize , sent_tokenize

stop_words = set(stopwords.words('english'))

from nltk import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

# filtered_words=[]
# for i in range(len(x_tokenized)):
#     for j in range(len(x_tokenized[i])):
#         words=x_tokenized[i][j]
#         filtered_words.append([word for word in words if word.lower() not in stop_words])

# filtered_words

sentences=x


for i in range(len(sentences)):
    words=word_tokenize(sentences[i])
    words=[word.lower() for word in words if word.lower() not in stop_words]
    sentences[i]=' '.join(words) # converting all the words into sentences



In [119]:
#sentences

In [120]:
import nltk
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize


lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

for i in range(len(sentences)):
    words = word_tokenize(sentences[i])
    pos_tags = pos_tag(words)
    lemmatized_words = [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in pos_tags]
    sentences[i] = ' '.join(lemmatized_words)




In [121]:
#sentences

In [122]:
import gensim
from gensim.models import Word2Vec, KeyedVectors

In [123]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

In [124]:
import numpy as np

def sentence_vector(sentence, wv):
    words = sentence.split()
    # Filter out words not in the model's vocabulary
    valid_words = [word for word in words if word in wv]
    if not valid_words:
        # If no valid words, return zero vector
        return np.zeros(wv.vector_size)
    # Average word vectors
    return np.mean(wv[valid_words], axis=0)


In [125]:
sentence_vectors = [sentence_vector(sent, wv) for sent in sentences]

In [126]:
from numpy.linalg import norm

def cosine_similarity(vec1, vec2):
    if norm(vec1) == 0 or norm(vec2) == 0:
        return 0
    return np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))


In [127]:
# def find_top5_similar(query, sentences, sentence_vectors, wv):
#     query_vec = sentence_vector(query, wv)
#     similarities = [cosine_similarity(query_vec, vec) for vec in sentence_vectors]
#     # Get indices of top 5 scores
#     top5_idx = np.argsort(similarities)[-5:][::-1]
#     return [(sentences[i], similarities[i]) for i in top5_idx]

def find_top5_similar(query, sentences, sentence_vectors, wv):
    query_vec = sentence_vector(query, wv)
    similarities = [cosine_similarity(query_vec, vec) for vec in sentence_vectors]
    # Get indices of top 5 scores
    top5_idx = np.argsort(similarities)[-5:][::-1]
    return [(i, sentences[i], similarities[i]) for i in top5_idx]

In [128]:
# query = "President got no money"
# results = find_top5_similar(query, sentences, sentence_vectors, wv)
# for sent, score in results:
#     print(f"{score:.4f} => {sent}")

query = "President got no money"
results = find_top5_similar(query, sentences, sentence_vectors, wv)

for idx, sent, score in results:
    print(f"{score:.4f} | Index: {idx} | Sentence: {sent}")



0.7130 | Index: 10045 | Sentence: guy 's run president want give 'free ' money
0.6888 | Index: 138279 | Sentence: money nothing
0.6834 | Index: 189577 | Sentence: get need little money
0.6436 | Index: 43264 | Sentence: really sure want president penny ?
0.6283 | Index: 150302 | Sentence: much money enough ?


In [129]:
x[150302]

'much money enough ?'